#   **Multimedia systems course | ECE AUTH | Winter semester 2022**
## **Simplified mp3 CODEC**

>   Giachoudis Christos


>   Kostopoulos Andreas Marios


![ logo](mp3.png)

# Necessary imports

In [1]:
# Importing the necessary libraries and modules
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.io import wavfile

## Import `wav` file

In [4]:
samplerate, data = wavfile.read('./myfile.wav')
print(f"WAV file imported successfully!!! This file has been created with a sample rate of {samplerate} Hz.")

WAV file imported successfully!!! This file has benn created with a sample rate of 44100 Hz.


In [6]:
# just for testing | at the end we can delete it
print(f"We will work with {type(data)}")

We will work with <class 'numpy.ndarray'>


## CODER

## DECODER